In [205]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import os  
os.getcwd()

'C:\\python\\파란학기\\수원시 평생학습관_데이터분석'

In [ ]:
종속변수 : 이탈 여부 (이탈 or 유지 )
독립변수 : 성별, 학년, 수원시평생학습관과의 거리 

In [1]:
### 데이터셋 불러오기 

In [2]:
### 데이터 셋 파악하기 

In [2]:
### 데이터 전처리   
- 파생변수 생성 ( [이탈_여부] 생성 )  
- 결측치 확인 
- 이상치 확인   
- 범주형 데이터 -> one hot encoding 으로 
- train/test set 나누기  
- 수치형 데이터 대해 정규화 or 표준화 검증  
- 머신러닝 알고리즘 기반 모델 생성 3가지  
(로지스틱회귀분석= LR, 선형판별분석 = LDA, KNN, 결정의사나무 = CART, SVC(랜텀포레스트 = RF, XGBoost )  
- 모델 검증 및 평가해서 정확도 가장 높은거로 
- test data의 예측값을 파생변수로 추가해 csv저장 

SyntaxError: invalid syntax (<ipython-input-2-05d1205c3b39>, line 2)

In [36]:
import pandas as pd    
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sns  
from sklearn.metrics import confusion_matrix  

from sklearn.model_selection import train_test_split  # Train과 Test set 나누기 
from sklearn.preprocessing import StandardScaler  
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀분석 
from sklearn.preprocessing import RobustScaler, LabelEncoder 
from sklearn.pipeline import make_pipeline  
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV  
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix  

from sklearn.linear_model import LogisticRegression, Lasso, ElasticNet  
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.kernel_ridge import KernelRidge  
from sklearn.svm import SVC  
from sklearn.ensemble import IsolationForest, RandomForestClassifier, RandomForestRegressor  
from xgboost import XGBClassifier



In [262]:
df1 = pd.read_csv('data1.csv', index_col = 0)  
df2 = pd.read_csv('data2.csv', index_col = 0)
df = pd.concat([df1, df2])

In [263]:
df.drop(['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis = 1, inplace = True)   
df

,성별,연령대,거주지,위도,경도
학습자,,,,,
1,남,30대,경기 수원시 팔달구 매산로2가,37.266706,127.004891
2,여,50대,경기 수원시 영통구 원천동,37.273600,127.057137
3,남,60대,경기 수원시 팔달구 우만동,37.285344,127.033899
4,여,50대,경기 평택시 신장동,37.081436,127.050039
5,여,50대,경기 평택시 신장동,37.081436,127.050039
...,...,...,...,...,...
9997,여,50대,경기 용인시 수지구,37.322150,127.097437
9998,남,60대,경기 수원시 영통구 이의동,37.304240,127.047337
9999,남,60대,경기 수원시 팔달구 갓매산로,37.268174,127.004991


In [264]:
df.dropna(axis=0, inplace = True)  
df.isnull().sum()  

성별     0
연령대    0
거주지    0
위도     0
경도     0
dtype: int64

In [265]:
diff = list((a,b) for a,b in zip(df['위도'], df['경도']))  
diff    

[(37.2667055, 127.00489099999999),
 (37.273600200000004, 127.05713670000002),
 (37.28534439999999, 127.0338987),
 (37.0814357, 127.05003870000002),
 (37.0814357, 127.05003870000002),
 (37.2490526, 127.0249436),
 (37.3042402, 127.04733700000001),
 (37.271962200000004, 127.0134416),
 (37.2779044, 127.0397213),
 (37.2944971, 127.0275727),
 (37.28534439999999, 127.0338987),
 (37.220154, 126.95025890000001),
 (37.28534439999999, 127.0338987),
 (37.2533239, 127.01300259999998),
 (37.2635576, 127.047265),
 (37.2828266, 126.9987271),
 (37.300401, 127.0070848),
 (37.271457700000006, 127.02815530000001),
 (37.242003499999996, 126.9685412),
 (37.220154, 126.95025890000001),
 (37.2635576, 127.047265),
 (37.2567849, 127.20612549999998),
 (37.2490526, 127.0249436),
 (37.3035412, 127.0154125),
 (37.285292299999995, 127.0293611),
 (37.2447039, 126.9813283),
 (37.2397806, 127.07381170000001),
 (37.2524485, 127.07097209999999),
 (37.271457700000006, 127.02815530000001),
 (37.271962200000004, 127.0134416

In [266]:
df['거리'] = df[['위도', '경도']].apply(tuple, axis =1)   
df

,성별,연령대,거주지,위도,경도,거리
학습자,,,,,,
1,남,30대,경기 수원시 팔달구 매산로2가,37.266706,127.004891,"(37.2667055, 127.00489099999999)"
2,여,50대,경기 수원시 영통구 원천동,37.273600,127.057137,"(37.273600200000004, 127.05713670000002)"
3,남,60대,경기 수원시 팔달구 우만동,37.285344,127.033899,"(37.28534439999999, 127.0338987)"
4,여,50대,경기 평택시 신장동,37.081436,127.050039,"(37.0814357, 127.05003870000002)"
5,여,50대,경기 평택시 신장동,37.081436,127.050039,"(37.0814357, 127.05003870000002)"
...,...,...,...,...,...,...
9997,여,50대,경기 용인시 수지구,37.322150,127.097437,"(37.32215, 127.0974374)"
9998,남,60대,경기 수원시 영통구 이의동,37.304240,127.047337,"(37.3042402, 127.04733700000001)"
9999,남,60대,경기 수원시 팔달구 갓매산로,37.268174,127.004991,"(37.2681743, 127.00499109999998)"


In [267]:
suwon = []
for i in range(len(df.index)):
    suwon.append((37.29168735861721, 127.02789467783248))
suwon  

[(37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.02789467783248),
 (37.29168735861721, 127.027894677

In [268]:
from scipy.spatial import distance  
for i in range(len(df.index)):
    df.iloc[i,5] = distance.euclidean(diff[i], suwon[i])    

In [269]:
df

,성별,연령대,거주지,위도,경도,거리
학습자,,,,,,
1,남,30대,경기 수원시 팔달구 매산로2가,37.266706,127.004891,0.0339597
2,여,50대,경기 수원시 영통구 원천동,37.273600,127.057137,0.0343837
3,남,60대,경기 수원시 팔달구 우만동,37.285344,127.033899,0.00873392
4,여,50대,경기 평택시 신장동,37.081436,127.050039,0.211415
5,여,50대,경기 평택시 신장동,37.081436,127.050039,0.211415
...,...,...,...,...,...,...
9997,여,50대,경기 용인시 수지구,37.322150,127.097437,0.0759221
9998,남,60대,경기 수원시 영통구 이의동,37.304240,127.047337,0.0231426
9999,남,60대,경기 수원시 팔달구 갓매산로,37.268174,127.004991,0.0328243
